# 0. Configure Package Dependencies 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np 
import pandas as pd
import shutil
from glob import glob 
import os
import cv2
import gc    # garbage collection, we need to save all the RAM we can
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers import Flatten, Dense, AveragePooling2D, Dropout, BatchNormalization, Activation, Conv2D, MaxPool2D
from keras.models import Model
from keras.optimizers import RMSprop, SGD
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm_notebook,trange  # a minimalistic yet powerful and easy to use progress bar

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# 1. Import the Dataset 

In [ ]:
path = '../input/histopathologic-cancer-detection/'           # data files 
train_dir = path + 'train/'  # training data directory
test_dir = path + 'test/'    # testing data directory

train = pd.DataFrame({'path':glob(os.path.join(train_dir,'*.tif'))})
train['id'] = train.path.map(lambda x: x.split('/')[4].split('.')[0]) # split path by '/'
labels = pd.read_csv(path+'train_labels.csv')
train = train.merge(labels, on='id') # merge labels and path

test = pd.DataFrame({'path':glob(os.path.join(test_dir,'*.tif'))})
test['id'] = test.path.map(lambda x: x.split('/')[4].split('.')[0])  

# 2. Preview the Dataset 

In [ ]:
# Display the dimensions of the dataset.
rows = train.shape[0]
columns = train.shape[1]
print('Total Number of Features: ', columns)
print('Total Number of Instances: ', rows)
# Preview the first 10 instances.
train.head(10)

In [ ]:
# Display the dimensions of the dataset.
rows = test.shape[0]
columns = test.shape[1]
print('Total Number of Features: ', columns)
print('Total Number of Instances: ', rows)
# Preview the first 10 instances.
test.head(10)

In [ ]:
def load_data(N,df):
    '''
    This functions loads N images using the dataframe df
    '''
    #为每个image分配一个96*96*3的numpy array,像素范围是[0,255]
    X = np.zeros([N,96,96,3],dtype=np.uint8) 
    #将label列转为numpy array
    y = np.squeeze(df.as_matrix(columns=['label']))[0:N]  #移除掉1维的列
    #迭代读取图像,tdqm 用于在notebook显示进度条
    for i, row in tqdm_notebook(df.iterrows(), total=N):
        if i == N:
            break
        X[i] = cv2.imread(row['path'])
          
    return X,y

In [ ]:
# Load 10k images
N=10000
X,y = load_data(N,train)

# 3.Exploratory Data Analysis (EDA)

The purpose of this EDA is to

- Take a look at the images
- Understand the distribution of the two classes (no cancer cells / cancer cells)
- Have a look at some image features (RGB channel distributions, mean brightness)

In [ ]:
# 展示一些示例图像
fig = plt.figure(figsize=(10, 4), dpi=150)  # 宽10英寸,高4英寸,分辨率150
np.random.seed(100) #we can use the seed to get a different set of random images

for plotNr,idx in enumerate(np.random.randint(0,N,8)):  # 随机产生8幅图像
    ax = fig.add_subplot(2, 8//2, plotNr+1, xticks=[], yticks=[]) #add subplots
    plt.imshow(X[idx]) #plot image
    ax.set_title('Label: ' + str(y[idx])) #show the label corresponding to the image

In [ ]:
# 查看数据分布情况(数据不是均匀分布的话,可以考虑over- and undersampling)
fig = plt.figure(figsize=(4, 2),dpi=150)
plt.bar([1,0], [(y==0).sum(), (y==1).sum()]) #plot a bar chart of the label frequency
plt.xticks([1,0],["Negative (N={})".format((y==0).sum()),"Positive (N={})".format((y==1).sum())]) # ([索引],[labels])
plt.ylabel("# of samples")

In [ ]:
# 查看每个类标签的情况
positive_samples = X[y == 1]
negative_samples = X[y == 0]

In [ ]:
# 比较RGB每个信道的像素值分布情况
nr_of_bins = 256 #each possible pixel value will get a bin in the following histograms
fig,axs = plt.subplots(4,2,sharey=True,figsize=(8,8),dpi=150)  # 4行2列的图

#RGB channels
axs[0,0].hist(positive_samples[:,:,:,0].flatten(),bins=nr_of_bins,density=True)  # 0表示的是RED信道
axs[0,1].hist(negative_samples[:,:,:,0].flatten(),bins=nr_of_bins,density=True)  
axs[1,0].hist(positive_samples[:,:,:,1].flatten(),bins=nr_of_bins,density=True)  # 1表示的是GREEN信道
axs[1,1].hist(negative_samples[:,:,:,1].flatten(),bins=nr_of_bins,density=True)
axs[2,0].hist(positive_samples[:,:,:,2].flatten(),bins=nr_of_bins,density=True)  # 2表示的是BLUE信道
axs[2,1].hist(negative_samples[:,:,:,2].flatten(),bins=nr_of_bins,density=True)

#All channels
axs[3,0].hist(positive_samples.flatten(),bins=nr_of_bins,density=True)
axs[3,1].hist(negative_samples.flatten(),bins=nr_of_bins,density=True)

#Set image labels
axs[0,0].set_title("Positive samples (N =" + str(positive_samples.shape[0]) + ")");
axs[0,1].set_title("Negative samples (N =" + str(negative_samples.shape[0]) + ")");
axs[0,1].set_ylabel("Red",rotation='horizontal',labelpad=35,fontsize=12)    # 设置Y轴标签,并转为水平
axs[1,1].set_ylabel("Green",rotation='horizontal',labelpad=35,fontsize=12)
axs[2,1].set_ylabel("Blue",rotation='horizontal',labelpad=35,fontsize=12)
axs[3,1].set_ylabel("RGB",rotation='horizontal',labelpad=35,fontsize=12)
for i in range(4):
    axs[i,0].set_ylabel("Relative frequency")
axs[3,0].set_xlabel("Pixel value")
axs[3,1].set_xlabel("Pixel value")
fig.tight_layout()    # 自动调整绘图窗口

- Negative samples seem to have higher, i.e. brighter, pixel values in general and especially in the green color channel.
- Interestingly, the positive samples have a darker green channel than red and blue while this is not true for the negative samples. However, very dark pixels are for both sample sets mostly only present in the green channel.
- Furthermore, note the relatively high frequency of the pixel value 255. Looking at the data above we can see, that these can likely be attributed to the bright white image regions present in some images. They seem to be present in both positive and negative samples similarly frequently.

In [ ]:
# 查看图像的平均亮度(图像像素值的平均数)
nr_of_bins = 64 #we use a bit fewer bins to get a smoother image
fig,axs = plt.subplots(1,2,sharey=True, sharex = True, figsize=(8,2),dpi=150)    # 1行2列
axs[0].hist(np.mean(positive_samples,axis=(1,2,3)),bins=nr_of_bins,density=True)
axs[1].hist(np.mean(negative_samples,axis=(1,2,3)),bins=nr_of_bins,density=True)
axs[0].set_title("Mean brightness, positive samples");
axs[1].set_title("Mean brightness, negative samples");
axs[0].set_xlabel("Image mean brightness")
axs[1].set_xlabel("Image mean brightness")
axs[0].set_ylabel("Relative frequency")
axs[1].set_ylabel("Relative frequency");

The distribution of mean brightness for the positive samples looks almost like a normal distribution around a brightness of 150. The negative samples, however, seem to follow some bimodal distribution with peaks around 140 and 225.  

#### Conclusions:  
- There are some easily spotted differences in the distributions of pixel values and mean image brightness between positive and negative samples. This is good, because whatever model we will use can likely use this.
- Some of the images seem to contain very bright regions, which are likely artifacts of the recording process. We might have to find a way to deal with them. They are almost equally distributed between positive and negative samples and, hence, probably not easily usable as a feature.
- We have about 50% more negative than positive samples. This might require adjustments.

# 4. Define the model 

In [ ]:
# 载入所有的data
N = train["path"].size # get the number of images in the training data set
X,y = load_data(N=N,df=train)

In [ ]:
# use the garbage collector and unbind some variables to free up space in our RAM.
# Collect garbage
positives_samples = None
negative_samples = None
gc.collect()

In [ ]:
# 分割data为训练集和验证集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
img_width = 96
img_height = 96
nbr_train_samples = len(X_train)
nbr_validation_samples = len(X_test)
batch_size = 32

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        rotation_range=10.,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True)

# this is the augmentation configuration we will use for validation:
# only rescaling
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(
        X_train, y_train,
#         target_size = (img_width, img_height),
        batch_size = batch_size,
        shuffle = True)

validation_generator = val_datagen.flow(
        X_test, y_test,
#         target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle = True)


In [ ]:
#just some network parameters, see above link regarding the layers for details
kernel_size = (3,3)
pool_size= (2,2)
first_filters = 32
second_filters = 64
third_filters = 128

#dropout is used for regularization here with a probability of 0.3 for conv layers, 0.5 for the dense layer at the end
dropout_conv = 0.3
dropout_dense = 0.5

#initialize the model
model = Sequential()

#now add layers to it

#conv block 1
model.add(Conv2D(first_filters, kernel_size, input_shape = (96, 96, 3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(first_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size)) 
model.add(Dropout(dropout_conv))

#conv block 2
model.add(Conv2D(second_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(second_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

#conv block 3
model.add(Conv2D(third_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(third_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

#a fully connected (also called dense) layer at the end
model.add(Flatten())
model.add(Dense(256, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(dropout_dense))

#finally convert to values of 0 to 1 using the sigmoid activation function
model.add(Dense(1, activation = "sigmoid"))

model.summary()

# 5. Evaluate the model 

In [ ]:
# Further, we will use binary crossentropy as loss function and the Adam optimizer. We set the learning rate of 0.001 for now.
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(0.001), 
              metrics=['accuracy'])

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size

model.fit_generator(
                train_generator,
                steps_per_epoch=STEP_SIZE_TRAIN,
                epochs=15,
                validation_data=validation_generator,
                validation_steps=STEP_SIZE_VALID)

In [ ]:
# make a prediction
y_pred_keras = model.predict_generator(test_gen, steps=len(df_val), verbose=1)
fpr_keras, tpr_keras, thresholds_keras = roc_curve(test_gen.classes, y_pred_keras)
auc_keras = auc(fpr_keras, tpr_keras)
auc_keras

In [ ]:
# Plot ROC Curve
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='area = {:.3f}'.format(auc_keras))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

# 6. Predict and submit "submission.csv" file

In [ ]:
X = None
y = None
gc.collect()

In [ ]:
test_files = glob(os.path.join(test_dir,'*.tif')) #find the test file names
submission = pd.DataFrame() #create a dataframe to hold results
file_batch = 5000 #we will predict 5000 images at a time
max_idx = len(test_files) #last index to use
for idx in range(0, max_idx, file_batch): #iterate over test image batches
    print("Indexes: %i - %i"%(idx, idx+file_batch))
    test_df = pd.DataFrame({'path': test_files[idx:idx+file_batch]}) #add the filenames to the dataframe
    test_df['id'] = test_df.path.map(lambda x: x.split('/')[3].split(".")[0]) #add the ids to the dataframe
    test_df['image'] = test_df['path'].map(cv2.imread) #read the batch
    K_test = np.stack(test_df["image"].values) #convert to numpy array
    predictions = model.predict(K_test,verbose = 1) #predict the labels for the test data
    test_df['label'] = predictions #store them in the dataframe
    submission = pd.concat([submission, test_df[["id", "label"]]])
    
submission.head() #display first lines

In [ ]:
submission.to_csv("submission.csv", index = False, header = True) #create the submission file